## Attempt 1 - Wrong Answer

In [ ]:
class Solution:
    cache = dict()
    def addOperators(self, num: str, target: int) -> List[str]:
        
        def evaluate(nums, ops):
            if tuple(nums + ops) in self.cache:
                return self.cache[tuple(nums + ops)]
            
            if not ops:
                return nums[0]
            
            L = len(ops)
            for idx in range(L):
                if ops[idx] == "*":
                    tmp = nums[idx] * nums[idx + 1]
                    new_nums, new_ops = nums[:], ops[:]
                    new_nums.pop(idx)
                    new_nums.pop(idx)
                    new_nums.insert(idx, tmp)
                    new_ops.pop(idx)
                    return evaluate(new_nums, new_ops)
            
            tmp = nums[0] + nums[1] if ops[0] == "+" else nums[0] - nums[1]
            new_nums, new_ops = nums[:], ops[:]
            new_nums.pop(0)
            new_nums.pop(0)
            new_nums.insert(0, tmp)
            new_ops.pop(0)
            return evaluate(new_nums, new_ops)
        
        def split(num):
            
            L = len(num)
            if (L == 1):
                return [([int(num)], [])]
            
            res = []
            for idx in range(1, L):
                if num[0] == '0' and idx > 1:
                    break
                pre, post = num[:idx], num[idx:]
                # if post[0] == '0' and len(post) > 1:
                #     continue
                
                rest = split(post)
                for pair in rest:
                    first = pair[0][:]
                    first.insert(0, int(pre))
                    for op in {'+', '-', '*'}:
                        second = pair[1][:]
                        second.insert(0, op)
                        res.append(tuple([first, second]))
            
            return res
        
        def join(nums, ops):
            L = len(ops)
            res = "".join([str(n) + op for n, op in zip(nums, ops)]) + str(nums[-1])
            return res
        
        splitted = split(num)
        # print(splitted)
        res = []
        for pair in splitted:
            if evaluate(*pair) == target:
                res.append(join(*pair))
        return res  

## Attempt 2 - Time Limit Exceeded

In [ ]:
from functools import partial

class Solution:
    def addOperators(self, num: str, target: int) -> List[str]:
        
        def mul(b, a=None):
            return a * b
        
        def ident(a):
            return a
        
        def evaluate(s):
            if '+' not in s and '-' not in s and '*' not in s:
                return int(s)
            
            res = 0
            operand = 0
            sign = 1
            func = ident
            
            for c in s:
                if c.isdigit():
                    operand = 10 * operand + int(c)
                
                if c == '+':
                    res += func(sign * operand)
                    operand = 0
                    sign = 1
                    func = ident
                
                if c == '-':
                    res += func(sign * operand)
                    operand = 0
                    sign = -1
                    func = ident
                
                if c == '*':
                    v = func(sign * operand)
                    operand = 0
                    sign = 1
                    func = partial(mul, a=v)
                
                # print(res)
            
            res += func(sign * operand)
            return res
        
        grow = [num[0]]
        L = len(num)
        for n in num[1:]:
            new_grow = []
            for g in grow:
                if g[-2:] not in {'+0', '-0', '*0'}:
                    new_grow.append(g + "" + n)
                new_grow.append(g + "+" + n)
                new_grow.append(g + "-" + n)
                new_grow.append(g + "*" + n)
            grow = new_grow
            
        candidates = []
        for g in grow:
            if '0' not in g:
                candidates.append(g)
                continue
                
            operands = []
            operand = ""
            for c in g:
                if c.isdigit():
                    operand += c
                else:
                    operands.append(operand)
                    operand = ""
            operands.append(operand)
            
            for o in operands:
                if ((int(o) == 0) and (len(o) > 1)) or ((int(o) != 0) and o.startswith('0')):
                    break
            else:
                candidates.append(g)
        
        res = []
        for g in candidates:
            if evaluate(g) == target:
                res.append(g)
        return res

## Lessons and Thoughts
- Attempt 1: 16/23 tests passed; Attempt 2: 15/23 tests passed. I believe Attempt 2 is correct, but checking for leading 0 probably taking too long. But then it is the same time complexity as given in the solution, ah well...
- Attempt 2 uses the evaluation method in Problem 772.